In [17]:
import numpy as np
import pandas as pd

# Idioms

## 满足某个条件下进行赋值

In [34]:
df = pd.DataFrame(
    {"AAA": [4,5,6,7], "BBB": [10,20,30,40], "CCC": [100,200,300,400], "DDD": [1,2,3,4]}
)
df

,AAA,BBB,CCC,DDD
0,4,10,100,1
1,5,20,200,2
2,6,30,300,3
3,7,40,400,4


首先需要明确df.loc的参数, 可以对多列进行条件判断, 或者赋值.

In [35]:
df.loc[(df.AAA >= 5) & (df.BBB <= 30), ["CCC", "DDD"]] = -1
df

,AAA,BBB,CCC,DDD
0,4,10,100,1
1,5,20,-1,-1
2,6,30,-1,-1
3,7,40,400,4


新建一列并根据条件赋值, 可以用np.where函数. 

In [20]:
df["logic"] = np.where(df.AAA > 5, "high", "low")
df

,AAA,BBB,CCC,logic
0,4,10,100,low
1,5,20,200,low
2,6,-1,-1,high
3,7,-1,-1,high


## 拆分